In [ ]:
# =========================================================
# SEÇÃO 1: IMPORTAÇÕES E SETUP GERAL
# =========================================================

In [ ]:
import os
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
from functools import partial

# --- Nossas funções auxiliares agora são importadas! ---
from helper.utils import (
    definir_seed,
    carregar_serie,
    dividir_serie_temporal,
    preparar_dados_para_neuralforecast
)

# --- Importações específicas para o treinamento ---
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, MLP, LSTM, Autoformer, NHITS

warnings.filterwarnings("ignore")

In [ ]:
# =========================================================
# SEÇÃO 2: PIPELINE DE EXPERIMENTO
# =========================================================

In [ ]:
def encontrar_melhor_arima_auto(treino_log, freq):
    # (Mantenha sua função original aqui)
    print("Buscando melhor ordem ARIMA com auto_arima...")
    m = 12 if freq.startswith('M') else (4 if freq.startswith('Q') else 1)
    auto_arima_model = pm.auto_arima(treino_log, m=m, seasonal=True, trace=False,
                                     error_action='ignore', suppress_warnings=True, stepwise=True)
    print(
        f"Melhor ordem encontrada: {auto_arima_model.order} Sazonal: {auto_arima_model.seasonal_order}")
    return auto_arima_model.order, auto_arima_model.seasonal_order

In [ ]:
def executar_experimento(nome_da_serie, horizonte):
    try:
        # --- Configurações Iniciais ---
        SEED = 42
        definir_seed(SEED)
        MAX_INPUT_SIZE = 24
        MAX_STEPS_NEURAL = 150

        # --- Carga e Preparação dos Dados ---
        serie_original = carregar_serie(nome_da_serie)
        percentual_treino = 1 - (horizonte / len(serie_original))
        if percentual_treino < 0.5:
            print(
                f"AVISO: Horizonte {horizonte} é muito grande para '{nome_da_serie}'. Pulando.")
            return None

        treino_orig, teste_orig = dividir_serie_temporal(
            serie_original, percentual_treino=percentual_treino)
        serie_log = np.log(serie_original)
        treino_log, _ = dividir_serie_temporal(
            serie_log, percentual_treino=percentual_treino)
        freq = serie_original.index.freqstr or pd.infer_freq(
            serie_original.index)
        if freq is None:
            return None

        previsoes_teste = {'y_true': teste_orig.values}

        # --- 1. Modelo ARIMA (Base para ambos os híbridos) ---
        modelo_arima = None
        try:
            print(f"Processando: ARIMA para '{nome_da_serie}'")
            ordem, ordem_sazonal = encontrar_melhor_arima_auto(
                treino_log, freq)
            modelo_arima = ARIMA(treino_log.asfreq(
                freq), order=ordem, seasonal_order=ordem_sazonal).fit()
            preds_log_teste_arima = modelo_arima.forecast(steps=horizonte)
            previsoes_teste['ARIMA'] = np.exp(preds_log_teste_arima).values
        except Exception as e:
            print(f"AVISO: ARIMA falhou para '{nome_da_serie}': {e}")
            return None


        # --- 2. Modelos Neurais Puros ---

        df_treino_log_nf = preparar_dados_para_neuralforecast(
            treino_log, nome_da_serie)

        modelos_para_testar = {'N-BEATS': NBEATS, 'MLP': MLP,
                               'LSTM': LSTM, 'Autoformer': Autoformer, 'N-HiTS': NHITS}


        for nome_modelo, classe_modelo in modelos_para_testar.items():

            try:
                print(f"Processando: {nome_modelo}")
                modelo_neural = [classe_modelo(input_size=min(
                    2 * horizonte, MAX_INPUT_SIZE), h=horizonte, max_steps=MAX_STEPS_NEURAL, scaler_type='standard', random_seed=SEED)]
                nf = NeuralForecast(models=modelo_neural, freq=freq)
                nf.fit(df=df_treino_log_nf, verbose=False)
                previsoes_teste[nome_modelo] = np.exp(
                    nf.predict()[classe_modelo.__name__].values)
            except Exception as e:
                print(f"AVISO: {nome_modelo} falhou: {e}")


        # Obter os resíduos do treino para os modelos híbridos

        residuos_treino_log = modelo_arima.resid

        # --- 3. Modelo Híbrido (MIMO) - Sua implementação original ---

        try:
            print("Processando: Híbrido (MIMO)")
            df_residuos_nf = preparar_dados_para_neuralforecast(
                residuos_treino_log, "residuos")

            # Treina um único modelo para prever todos os H passos
            modelo_residuos_mimo = [NBEATS(input_size=min(
                2*horizonte, MAX_INPUT_SIZE), h=horizonte, max_steps=MAX_STEPS_NEURAL, scaler_type='standard', random_seed=SEED)]

            nf_residuos_mimo = NeuralForecast(
                models=modelo_residuos_mimo, freq=freq)

            nf_residuos_mimo.fit(df=df_residuos_nf, verbose=False)

            preds_residuos_log_mimo = nf_residuos_mimo.predict()[
                'NBEATS'].values
            previsoes_teste['Híbrido (MIMO)'] = previsoes_teste['ARIMA'] + \
                preds_residuos_log_mimo
        except Exception as e:
            print(f"AVISO: Híbrido (MIMO) falhou: {e}")

        # =========================================================================================
        # --- 4. Híbrido (Recursive-Direct) - IMPLEMENTAÇÃO FINAL ROBUSTA ---
        # =========================================================================================
        try:
            print("Processando: Híbrido (Recursive-Direct)")

            # --- 4.1 Preparação dos dados em formato Painel ---
            lista_dfs_horizontes = []
            input_size_nbeats = min(2 * horizonte, MAX_INPUT_SIZE)

            for h_step in range(1, horizonte + 1):
                # Cria um df para o horizonte específico
                df_residuos_h = residuos_treino_log.to_frame(name='y')
                df_residuos_h['ds'] = df_residuos_h.index
                df_residuos_h['y'] = df_residuos_h['y'].shift(-h_step + 1)

                # O ID único agora é o próprio horizonte
                df_residuos_h['unique_id'] = f'h_{h_step}'

                df_residuos_h.dropna(inplace=True)

                # Só adiciona o df se ele tiver dados suficientes
                if len(df_residuos_h) >= input_size_nbeats + 1:
                    lista_dfs_horizontes.append(df_residuos_h)

            if not lista_dfs_horizontes:
                raise ValueError(
                    "Nenhum horizonte tinha dados suficientes para treinar o modelo de resíduos.")

            # Concatena todos os dataframes em um só (formato painel)
            df_treino_residuos_painel = pd.concat(lista_dfs_horizontes)

            # --- 4.2 Treinamento e Previsão ---
            # Treina um único modelo N-BEATS que aprenderá com todos os horizontes (unique_ids)
            modelo_residuos_direct = [NBEATS(
                input_size=input_size_nbeats, h=1, max_steps=MAX_STEPS_NEURAL, scaler_type='standard', random_seed=SEED)]
            nf_residuos_direct = NeuralForecast(
                models=modelo_residuos_direct, freq=freq)

            # O fit é feito uma única vez no dataframe de painel
            nf_residuos_direct.fit(df=df_treino_residuos_painel, verbose=False)

            # O predict irá gerar uma previsão para cada unique_id (cada horizonte)
            preds_df = nf_residuos_direct.predict()

            # Ordena as previsões pela ordem do horizonte para garantir a sequência correta
            preds_df['h_step'] = preds_df['unique_id'].str.replace(
                'h_', '').astype(int)
            preds_df.sort_values(by='h_step', inplace=True)

            preds_residuos_log_direct = preds_df['NBEATS'].values

            # Garantir que o array de predições tenha o tamanho do horizonte, preenchendo com 0 se algum especialista não foi treinado
            if len(preds_residuos_log_direct) < horizonte:
                preds_completas = np.zeros(horizonte)
                # h_1 está no índice 0
                indices_validos = preds_df['h_step'].values - 1
                preds_completas[indices_validos] = preds_residuos_log_direct
                preds_residuos_log_direct = preds_completas

            # --- 4.3 Combinação Final ---
            previsoes_teste['Híbrido (Recursive-Direct)'] = previsoes_teste['ARIMA'] + \
                preds_residuos_log_direct

        except Exception as e:
            import traceback
            print(f"AVISO: Híbrido (Recursive-Direct) falhou: {e}")
            traceback.print_exc()
            # Garante que a coluna exista com NaNs para não quebrar a estrutura final
            previsoes_teste['Híbrido (Recursive-Direct)'] = np.nan

        # --- Finalização ---
        df_final = pd.DataFrame(previsoes_teste, index=teste_orig.index)
        df_final['dataset'] = nome_da_serie
        df_final['horizonte'] = horizonte
        return df_final.reset_index().rename(columns={'index': 'ds'})

    except Exception as e:
        import traceback
        print(
            f"ERRO GERAL no processamento de '{nome_da_serie}' para o horizonte {horizonte}: {e}")
        traceback.print_exc()
        return None

In [ ]:
# =========================================================
# SEÇÃO 3: ORQUESTRADOR (LÓGICA DA CAMADA SILVER)
# =========================================================

In [ ]:
# --- Configurações ---
LISTA_DE_DATASETS = ['AirPassengers', 'co2', 'nottem', 'austres', 'lynx']
VETOR_DE_HORIZONTES = [10, 12, 15, 24]
output_dir = "./data/silver"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "resultados_completos.csv")

# --- Lógica de Execução Incremental ---
# Apaga o arquivo antigo para garantir uma execução limpa
if os.path.exists(output_file):
    os.remove(output_file)

header_escrito = False

In [ ]:
for dataset in tqdm(LISTA_DE_DATASETS, desc="Processando Datasets"):
    for horizonte in tqdm(VETOR_DE_HORIZONTES, desc=f"Testando Horizontes para {dataset}", leave=False):
        # Executa o experimento para uma combinação de dataset e horizonte
        df_resultado_detalhado = executar_experimento(dataset, horizonte)

        if df_resultado_detalhado is not None:
            # Lógica para salvar incrementalmente no arquivo da camada Silver
            if not header_escrito:
                # Escreve o cabeçalho apenas na primeira vez
                df_resultado_detalhado.to_csv(
                    output_file, index=False, mode='w', header=True)
                header_escrito = True
            else:
                # Anexa os novos resultados sem o cabeçalho
                df_resultado_detalhado.to_csv(
                    output_file, index=False, mode='a', header=False)

print(f"\nProcesso finalizado. Resultados salvos em '{output_file}'")